In [147]:
import os
import fastf1 as ff1
import pandas as pd 
import numpy as np
import pandas as pd

Getting details from 2024 Bahrain Grand Prix

In [ ]:
if not os.path.exists('./f1_cache'):
    os.makedirs('./f1_cache')
    ff1.Cache.enable_cache('./f1_cache')

session = ff1.get_session(2024, 'Bahrain', 'R')
session.load()

In [162]:
laps2024 = session.laps[['Driver', 'LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']].copy()
laps2024.dropna(inplace=True)
# Convert columns to timedelta
laps2024['LapTime'] = pd.to_timedelta(laps2024['LapTime'])
laps2024['Sector1Time'] = pd.to_timedelta(laps2024['Sector1Time'])
laps2024['Sector2Time'] = pd.to_timedelta(laps2024['Sector2Time'])
laps2024['Sector3Time'] = pd.to_timedelta(laps2024['Sector3Time'])

# Convert timedelta to total seconds
laps2024['LapTime'] = laps2024['LapTime'].dt.total_seconds()
laps2024['Sector1Time'] = laps2024['Sector1Time'].dt.total_seconds()
laps2024['Sector2Time'] = laps2024['Sector2Time'].dt.total_seconds()
laps2024['Sector3Time'] = laps2024['Sector3Time'].dt.total_seconds()

laps2024 = laps2024.groupby('Driver')[['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']].mean().reset_index()

laps2024.head()

,Driver,LapTime,Sector1Time,Sector2Time,Sector3Time
0,ALO,89.363364,23.049894,46.363879,19.949591
1,BEA,89.203727,23.019818,46.638561,19.545348
2,BOT,89.349485,22.794742,46.705591,19.849152
3,COL,89.942000,23.702310,46.814586,19.425103
4,GAS,88.297000,22.381167,46.361242,19.554591


Getting details from 2025 Bahrain Qualifying

In [170]:
qualifying_2025 = pd.DataFrame({
    "Driver": ["Oscar Piastri", "George Russell", "Lando Norris", "Max Verstappen", "Lewis Hamilton",
               "Charles Leclerc", "Isack Hadjar", "Kimi Antonelli", "Yuki Tsunoda", "Alexander Albon",
               "Esteban Ocon", "Nico Hülkenberg", "Fernando Alonso", "Lance Stroll", "Carlos Sainz Jr.",
               "Pierre Gasly", "Oliver Bearman", "Jack Doohan", "Gabriel Bortoleto", "Liam Lawson"],
    "QualifyingTime (s)": [89.841, 90.009, 90.267, 90.423, 90.772,
                           90.175, 91.271, 90.213, 91.228, 91.706,
                           92.040, 92.067, 91.688, 92.283, 90.680 ,
                           90.216, 92.373, 91.245, 92.186, 92.165]
})

In [169]:
# Map full names to FastF1 3-letter codes
driver_mapping = {
    "Oscar Piastri": "PIA", "George Russell": "RUS", "Lando Norris": "NOR", "Max Verstappen": "VER",
    "Lewis Hamilton": "HAM", "Charles Leclerc": "LEC", "Isack Hadjar": "HAD", "Kimi Antonelli": "ANT",
    "Yuki Tsunoda": "TSU", "Alexander Albon": "ALB", "Esteban Ocon": "OCO", "Nico Hülkenberg": "HUL",
    "Fernando Alonso": "ALO", "Lance Stroll": "STR", "Carlos Sainz Jr.": "SAI", "Pierre Gasly": "GAS",
    "Oliver Bearman": "BEA", "Jack Doohan": "DOO", "Gabriel Bortoleto": "BOR", "Liam Lawson": "LAW"
}

In [171]:
qualifying_2025["DriverCode"] = qualifying_2025["Driver"].map(driver_mapping)

In [172]:
qualifying_2025.head()


,Driver,QualifyingTime (s),DriverCode
0,Oscar Piastri,89.841,PIA
1,George Russell,90.009,RUS
2,Lando Norris,90.267,NOR
3,Max Verstappen,90.423,VER
4,Lewis Hamilton,90.772,HAM


In [173]:
merged_data = qualifying_2025.merge(laps2024, left_on="DriverCode", right_on="Driver", how="left")
merged_data.head()


,Driver_x,QualifyingTime (s),DriverCode,Driver_y,LapTime,Sector1Time,Sector2Time,Sector3Time
0,Oscar Piastri,89.841,PIA,PIA,88.642530,22.811258,46.239667,19.591606
1,George Russell,90.009,RUS,RUS,88.477091,22.763909,46.251364,19.461818
2,Lando Norris,90.267,NOR,NOR,88.550742,22.752909,46.229106,19.568727
3,Max Verstappen,90.423,VER,VER,87.882273,22.346712,46.152061,19.383500
4,Lewis Hamilton,90.772,HAM,HAM,88.982530,22.777121,46.489803,19.715606
